In [1]:
import pystac_client
import odc.stac
from dask.distributed import Client, LocalCluster
import rioxarray
from odc.algo import to_f32, xr_geomedian

# Setup

In [2]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [3]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [4]:
try:
    cluster.close()
except:
    pass
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 63.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51509,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:51543,Total threads: 4
Dashboard: http://127.0.0.1:51546/status,Memory: 10.65 GiB
Nanny: tcp://127.0.0.1:51512,


In [5]:
# Area bounds - map derived
bounds = (122.1767, -18.2329, 123.1410, -17.7193)

# Load Sentinel-2 Data

In [6]:
bands = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_red_edge_1', 'nbart_red_edge_2', 'nbart_red_edge_3', 'nbart_nir_1', 'nbart_nir_2', 'nbart_swir_2', 'nbart_swir_3']
band_labels = ['Blue', 'Green', 'Red', 'RE 1', 'RE 2', 'RE 3', 'NIR 1', 'NIR 2', 'SWIR-2', 'SWIR-3']

In [7]:
start_date = "2025-07-01"
end_date = "2025-07-31"

In [8]:
query = catalog.search(
    bbox=bounds,
    collections=["ga_s2am_ard_3", "ga_s2bm_ard_3"],
    datetime=f"{start_date}/{end_date}"
)
items = list(query.items())

In [9]:
dask_chunk_size = 1000

In [10]:
ds = odc.stac.load(
    items=items,
    bands=bands,
    groupby="solar_day",
    bbox=bounds,
    chunks={"y":dask_chunk_size, "x":dask_chunk_size}
)

In [11]:
# Scale values using the to_f32 util function
sr_max_value = 10000
scale, offset = (1 / sr_max_value, 0)
ds_scaled = to_f32(ds, scale=scale, offset=offset)

In [12]:
ds_median = xr_geomedian(ds_scaled, num_threads=1, eps=1e-7).rio.write_crs(ds.rio.crs)   # Need to write CRS because geomedian removes it

In [13]:
ds_median = ds_median.compute()

## Save data

In [14]:
ds_median.to_zarr('ds_median_Yawuru_2025_07.zarr', mode='w')